#CBT Cognitive Distortion & Sentiment Analyzer

This Python script leverages pre-trained language models and regex patterns to help users identify common cognitive distortions in their thoughts and understand the overall sentiment expressed.

##Features:

Cognitive Distortion Detection: Uses regular expressions to detect classic thinking errors like all-or-nothing thinking, catastrophizing, personalization, and more.

Sentiment Analysis: Employs a pre-trained DistilBERT model fine-tuned for sentiment classification to automatically label inputs as positive or negative with confidence scores.

Refined Emotional Reasoning Detection: Special handling for “I feel...” statements, ignoring neutral feelings such as “okay” or “fine” to reduce false positives.

Interactive User Input: Accepts user thoughts interactively with an option to exit the program.

Reframe Suggestions: Provides helpful suggestions to rethink and challenge detected distortions.

Importing dependencies...

In [ ]:
!pip install transformers -q

In [ ]:
import re
from transformers import pipeline

Below we load a pretrained DistilBERT model fine-tuned on SST-2 dataset for binary sentiment classification (positive/negative).

Runs locally without requiring API keys.

In [ ]:
# Load the emotion classifier pipeline
emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)


##Define Cognitive Distortion Patterns and Suggestions

`distortion_patterns`: Dictionary where each cognitive distortion maps to a list of regex patterns to detect it.

`reframe_suggestions`: Helpful advice to challenge and reframe distorted thinking.

`neutral_feelings`: List of words considered neutral feelings to avoid false flags in emotional reasoning.

In [ ]:
# Load pre-trained sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Cognitive distortion patterns (your updated version)
distortion_patterns = {
    "All-or-Nothing Thinking": [
        r"\balways\b",
        r"\bnever\b",
        r"\bcompletely\b",
        r"\btotally\b",
        r"\beveryone\b",
        r"\bnobody\b",
        r"\b(can't|cannot) do anything\b",
        r"\b(can't|cannot) .* right\b",
        r"\b(none|nothing) works\b"
    ],
    "Catastrophizing": [
        r"\bworst\b",
        r"\bdisaster\b",
        r"\bruined\b",
        r"\bcannot handle\b",
        r"\beverything is\b",
        r"\bgoing to fail\b"
    ],
    "Overgeneralization": [
        r"\beveryone\b",
        r"\bno one\b",
        r"\balways\b",
        r"\bnever\b",
        r"\b(can't|cannot) do anything\b"
    ],
    "Mental Filter": [
        r"\bonly\b",
        r"\bjust\b",
        r"\balways\b"
    ],
    "Should Statements": [
        r"\bshould\b",
        r"\bmust\b",
        r"\bhave to\b"
    ],
    "Personalization": [
        r"\bit'?s my fault\b",
        r"\bi caused\b",
        r"\bit'?s because of me\b"
    ],
    "Mind Reading": [
        r"\bthey think\b",
        r"\bthey believe\b",
        r"\bthey don'?t like\b",
        r"\bpeople think\b"
    ],
    "Labeling": [
        r"\bi'?m a loser\b",
        r"\bhe'?s a failure\b",
        r"\bshe'?s useless\b",
        r"\bi am worthless\b",
        r"\bi'?m worthless\b"
    ],
    "Magnification / Minimization": [
        r"\bexaggerate\b",
        r"\btrivialize\b",
        r"\boverreact\b",
        r"\bmake a big deal\b"
    ]
}

reframe_suggestions = {
    "All-or-Nothing Thinking": "Look for shades of gray. Are there exceptions or partial truths?",
    "Catastrophizing": "Consider more realistic, less extreme outcomes.",
    "Overgeneralization": "Avoid sweeping conclusions; focus on specific facts.",
    "Mental Filter": "Try to see the whole picture, not just the negatives.",
    "Should Statements": "Replace 'should' with preferences or choices to reduce pressure.",
    "Personalization": "Consider other factors beyond yourself that may have contributed.",
    "Mind Reading": "Avoid assuming you know what others think; check the facts.",
    "Emotional Reasoning": "Feelings are valid but aren’t always facts; look for evidence.",
    "Labeling": "Focus on specific behaviors rather than global labels.",
    "Magnification / Minimization": "Try to keep problems and successes in balanced perspective."
}

neutral_feelings = {"okay", "fine", "good", "alright", "well", "normal", "decent", "not bad"}


Below is the code for detecting emotional reasoning and cognitive distortions:

In [ ]:
def detect_emotional_reasoning(text):
    text = text.lower()
    pattern = r"\bi feel (\w+)"
    matches = re.findall(pattern, text)
    for feeling in matches:
        if feeling not in neutral_feelings:
            return True
    return False

def detect_distortions(text):
    text = text.lower()
    found_distortions = {}

    for distortion, patterns in distortion_patterns.items():
        if distortion == "Emotional Reasoning":
            continue

        matches = []
        for pattern in patterns:
            if re.search(pattern, text):
                display_pattern = pattern.replace(r"\b", "").replace(r"(?:", "").replace(r")", "").replace(r"\\", "")
                matches.append(display_pattern)
        if matches:
            found_distortions[distortion] = matches

    # Emotional Reasoning separately
    if detect_emotional_reasoning(text):
        found_distortions["Emotional Reasoning"] = ["i feel <negative feeling>"]

    return found_distortions

def analyze_cbt_thought(text):
    distortions_found = detect_distortions(text)
    if distortions_found:
        print("\nCognitive Distortions Detected:")
        for dist, matches in distortions_found.items():
            print(f"\n- {dist}:")
            print(f"  Trigger phrases matched: {', '.join(matches)}")
            print(f"  Reframe suggestion: {reframe_suggestions.get(dist, 'Try to rethink this thought.')}")
    else:
        print("\nNo common cognitive distortions detected. Keep it balanced!")

def analyze_with_sentiment(text):
    # Sentiment analysis using pre-trained model
    result = sentiment_analyzer(text)[0]
    label = result['label']   # 'POSITIVE' or 'NEGATIVE'
    score = result['score']

    print(f"\nSentiment Analysis: {label} (Confidence: {score:.2f})")

    # Cognitive distortions check
    analyze_cbt_thought(text)



Testing user input:

In [ ]:
print("Welcome to the CBT Cognitive Distortion + Sentiment Analyzer!")
print("Type your thought below and press Enter. Type 'exit' to quit.\n")

while True:
    user_input = input("Enter your thought: ")
    if user_input.strip().lower() == "exit":
        print("Goodbye! Take care.")
        break
    analyze_with_sentiment(user_input)
